<!--NAVIGATION-->
< [EnergyPlus Scripting Using Archetypal](02.01-EnergyPlus-Scripting-Using-Archetypal.ipynb) | [Contents](Index.ipynb) | [Parsing an Existing Model](02.03-Parsing-an-Existing-Model.ipynb) >

# Getting and Setting Model Attibutes

The main use case for archetypal is the ability to programatically modify component attributes.

In [1]:
from archetypal import IDF

idf = IDF()

/Users/samueld/PycharmProjects/surrogate-api/.venv/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.5, the latest is 0.4.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


To access all of the Material objects in the IDF model:

In [2]:
idf.idfobjects["MATERIAL"]

[]

The output above shows that the returns list is empty, since our empty model does not have an materials in it. Let's create one.

There are different ways of adding new objects to a model:

- `IDF.newidfobject` [docs](https://archetypal.readthedocs.io/en/stable/reference/archetypal.idfclass.IDF.html#archetypal.idfclass.IDF.newidfobject) will create a new component and add it directly to the model.
- `IDF.anidfobject` [docs](https://archetypal.readthedocs.io/en/stable/reference/archetypal.idfclass.IDF.html#archetypal.idfclass.IDF.anidfobject) in combination with `IDF.addidfobject` [docs](https://archetypal.readthedocs.io/en/stable/reference/archetypal.idfclass.IDF.html#archetypal.idfclass.IDF.addidfobject) and `IDF.addidfobjects` [docs](https://archetypal.readthedocs.io/en/stable/reference/archetypal.idfclass.IDF.html#archetypal.idfclass.IDF.addidfobjects) will create a new component but will not be added to the model `IDF.addidfobject` or `IDF.addidfobjects` must be called after the object creation.

For example, we specify a new Material and its properties by passnig the `key` (or the objects Component type) as an All-Caps string "MATERIAL" and then the attributes as key-value pairs like in any python methods or functions:

In [3]:
idf.newidfobject("MATERIAL", Name="My New Material", Thickness=0.05)


MATERIAL,
    My New Material,          !- Name
    ,                         !- Roughness
    0.05,                     !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

We can verify that this object has been added by looking at the dictionary repository of objects defined in the model using the `IDF.idfobjects` property:

In [4]:
idf.idfobjects["MATERIAL"]

[
MATERIAL,
    My New Material,          !- Name
    ,                         !- Roughness
    0.05,                     !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
]

Similarly, we can delay the craetion of an object by calling `IDF.anidfobject` instead:

In [5]:
mat = idf.anidfobject(
    "MATERIAL", Name="My New Material 2", Thickness=0.05, Thermal_Absorptance=0.89
)
mat  # object has been assigned to variable `mat`


MATERIAL,
    My New Material 2,        !- Name
    ,                         !- Roughness
    0.05,                     !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.89,                     !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

And then the object can be added to the model with:

In [6]:
idf.addidfobject(mat)


MATERIAL,
    My New Material 2,        !- Name
    ,                         !- Roughness
    0.05,                     !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.89,                     !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

Calling again `IDF.idfobjects`, we have 2 materials in the model:

In [7]:
idf.idfobjects["MATERIAL"]

[
MATERIAL,
    My New Material,          !- Name
    ,                         !- Roughness
    0.05,                     !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
, 
MATERIAL,
    My New Material 2,        !- Name
    ,                         !- Roughness
    0.05,                     !- Thickness
    ,                         !- Conductivity
    ,                         !- Density
    ,                         !- Specific Heat
    0.89,                     !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance
]

### Tips and tricks

Granted it can be hard to know what are the attribute names of a component especially since EnergyPlus has a huge list of components to choose from.

One trick is to call the creation of an object with it's default value and to call the `dict` methdod to return the atribute names:

In [8]:
idf.anidfobject("MATERIAL:NOMASS").to_dict()

{'key': 'MATERIAL:NOMASS',
 'Name': '',
 'Roughness': '',
 'Thermal_Resistance': '',
 'Thermal_Absorptance': 0.9,
 'Solar_Absorptance': 0.7,
 'Visible_Absorptance': 0.7}

You would then copy the dictionary above in a cell and make changes to the attribute values as such:

In [9]:
idf.anidfobject("MATERIAL:NOMASS").getfieldidd("Roughness")

{'field': ['Roughness'],
 'required-field': [''],
 'type': ['choice'],
 'key': ['VeryRough',
  'Rough',
  'MediumRough',
  'MediumSmooth',
  'Smooth',
  'VerySmooth']}

In [10]:
mat_dict = {
    "key": "MATERIAL:NOMASS",
    "Name": "My NoMass material",
    "Roughness": "",
    "Thermal_Resistance": "",
    "Thermal_Absorptance": 0.9,
    "Solar_Absorptance": 0.7,
    "Visible_Absorptance": 0.7,
}

But what is "**Roughness**"! And what are the valid values that EnergyPlus will accept? We could go to the EnergyPlus online documentation for the Material:NoMass class and fetch the info there, but there is an easier option:

Thankfully, `eppy` has done some heavylifting behind the scene and has parsed all of the content of the EnergyPlus IDD file which holds the recipy book for all the EnergyPlus component in a specific EnergyPlus version.

One can retrieve this IDD information for any fieldname by calling `EPBunch.getfield(<fieldname>)`.

<div class="alert alert-block alert-info">
<b>Sidenote:</b> the components in an IDF model are of a special type of class called an `EPBunch`.
</div>

In [11]:
idf.anidfobject("MATERIAL:NOMASS").getfieldidd("Roughness")

{'field': ['Roughness'],
 'required-field': [''],
 'type': ['choice'],
 'key': ['VeryRough',
  'Rough',
  'MediumRough',
  'MediumSmooth',
  'Smooth',
  'VerySmooth']}

The information above tells me that the field "Roughness" is not a "required-field" (it can be left empty), and that the choices are on off ['VeryRough',  'Rough',  'MediumRough',  'MediumSmooth',  'Smooth',  'VerySmooth']:

So I can continue edition my dictionary with the required Roughness:

In [12]:
mat_dict = {
    "key": "MATERIAL:NOMASS",
    "Name": "My NoMass material",
    "Roughness": "Smooth",
    "Thermal_Resistance": "",
    "Thermal_Absorptance": 0.9,
    "Solar_Absorptance": 0.7,
    "Visible_Absorptance": 0.7,
}

Finally, I can create the new material object using the dictionary and python's special double-star (**) syntax which will unpack the key-values of the dictionary as paramters of the `newidfobject` method: 

In [13]:
idf.newidfobject(**mat_dict)


MATERIAL:NOMASS,
    My NoMass material,       !- Name
    Smooth,                   !- Roughness
    ,                         !- Thermal Resistance
    0.9,                      !- Thermal Absorptance
    0.7,                      !- Solar Absorptance
    0.7;                      !- Visible Absorptance

<!--NAVIGATION-->
< [EnergyPlus Scripting Using Archetypal](02.01-EnergyPlus-Scripting-Using-Archetypal.ipynb) | [Contents](Index.ipynb) | [Parsing an Existing Model](02.03-Parsing-an-Existing-Model.ipynb) >